Link to Brijesh's notebook for text and documentation:

https://colab.research.google.com/drive/1bA6z93JB-g1U2Mz6-YAdqpmoqsZBITdv?resourcekey=0-66KLAWuQZTLgNewtVIoLvA&usp=sharing

# ↘️ Get the code

https://github.com/brijk7/python-docs-samples/tree/main/people-and-planet-ai/rain-nowcasting

In [4]:
# TODO: REPLACE WITH URL FROM UPSTREAM MAIN
# repo_url = "https://raw.githubusercontent.com/GoogleCloudPlatform/python-docs-samples/ppai-weather-forecasting/people-and-planet-ai/weather-forecasting"
repo_url = "https://raw.githubusercontent.com/davidcavazos/python-docs-samples/ppai-weather-forecasting/people-and-planet-ai/weather-forecasting"

!wget --quiet -O requirements.txt {repo_url}/requirements.txt
!wget --quiet -O constraints.txt {repo_url}/constraints.txt

In [ ]:
!pip --quiet install --upgrade pip

In [ ]:
!pip --quiet install -r requirements.txt -c constraints.txt google-cloud-aiplatform

# Restart the runtime by ending the runtime's process
exit()

## ⚠️ RESTART RUNTIME ⚠️

# ✏️ Enter your Cloud project's details

Ensure you provide a regional Google Cloud Storage bucket! This is needed for Vertex AI.

In [1]:
import os
from google.colab import auth

auth.authenticate_user()

# Please fill in these values.
project = "dcavazos-lyra" #@param {type:"string"}
bucket = "dcavazos-lyra" #@param {type:"string"}
location = "us-central1" #@param {type:"string"}

# Load values from environment variables if available.
project = os.environ.get("GOOGLE_CLOUD_PROJECT", project)
bucket = os.environ.get("CLOUD_STORAGE_BUCKET", bucket)
location = os.environ.get("CLOUD_LOCATION", location)

# Quick input validations.
assert project, "⚠️ Please provide a Google Cloud project ID"
assert bucket, "⚠️ Please provide a Cloud Storage bucket name"
assert not bucket.startswith('gs://'), f"⚠️ Please remove the gs:// prefix from the bucket name: {bucket}"
assert location, "⚠️ Please provide a Google Cloud location"

# Configure gcloud.
!gcloud config set project {project}
os.environ["GOOGLE_CLOUD_PROJECT"] = project
assert False, project

Updated property [core/project].


In [58]:
# TODO: REMOVE THESE CLEANUP STEPS
%cd /content
!rm -rf python-docs-samples

# TODO: REPLACE WITH URL FROM UPSTREAM MAIN
# !git clone https://github.com/GoogleCloudPlatform/python-docs-samples.git
!git clone -b ppai-weather-forecasting https://github.com/davidcavazos/python-docs-samples.git
%cd python-docs-samples/people-and-planet-ai/weather-forecasting

# TODO: REMOVE THIS
try:
  import datasets
  import trainer
  import importlib
  importlib.reload(datasets)
  importlib.reload(trainer.task)
except:
  pass

/content
Cloning into 'python-docs-samples'...
remote: Enumerating objects: 66982, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 66982 (delta 112), reused 157 (delta 101), pack-reused 66813
Receiving objects: 100% (66982/66982), 104.96 MiB | 36.96 MiB/s, done.
Resolving deltas: 100% (38873/38873), done.
/content/python-docs-samples/people-and-planet-ai/weather-forecasting


# 🗺️ Authenticate to Earth Engine

TODO: add link to the `datasets.py` source file

In [3]:
import ee
import google.auth

credentials, _ = google.auth.default()
ee.Initialize(
    credentials.with_quota_project(None),
    project=project,
    opt_url="https://earthengine-highvolume.googleapis.com",
)

# 🛰️ Get the training data

The training data for the model is collected by sampling the GOES-16 and GPM
datasets in Earth Engine. We will first define some global constants for convenience.

https://developers.google.com/earth-engine/datasets/catalog/NOAA_GOES_16_MCMIPF

https://developers.google.com/earth-engine/datasets/catalog/NASA_GPM_L3_IMERG_V06

https://en.wikipedia.org/wiki/Hurricane_Dorian

In [4]:
from datetime import datetime
import folium

from datasets import get_label_image

image = get_label_image(datetime(2019, 9, 3, 18))
mask = image.gt(0.1)
image = image.updateMask(mask)
vis_params = {
    "min": 0.0,
    "max": 15.0,
    "palette": [
        '000096','0064ff', '00b4ff', '33db80', '9beb4a',
        'ffeb00', 'ffb300', 'ff6400', 'eb1e00', 'af0000'
    ],
}
map = folium.Map(location=(25, -90), zoom_start=5)
folium.TileLayer(
    name="Global Precipitation Measurement (GPM)",
    tiles=image.getMapId(vis_params)["tile_fetcher"].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay = True,
).add_to(map)
folium.LayerControl().add_to(map)
map

In [9]:
from datetime import datetime
import folium

from datasets import get_input_image

image = get_input_image(datetime(2019, 9, 3, 18))
vis_params = {
    "bands": ['CMI_C02', 'CMI_C03', 'CMI_C01'],
    "min": 0.0,
    "max": 3000.0,
}
folium.Map(
    location=(25, -90),
    zoom_start=5,
    tiles=image.getMapId(vis_params)["tile_fetcher"].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
)

## Create datasets in Dataflow

In [ ]:
!gsutil -m rm -rf gs://{bucket}/weather/data

In [ ]:
!python datasets.py \
  --output-path "gs://{bucket}/weather/data-small" \
  --num-dates "10" \
  --num-points "5" \
  --runner "DataflowRunner" \
  --project "{project}" \
  --region "{location}" \
  --temp_location "gs://{bucket}/weather/temp"

https://console.cloud.google.com/dataflow/jobs

In [ ]:
# !gsutil du -sh gs://dcavazos-lyra/weather/data-20

1.18 GiB     gs://dcavazos-lyra/weather/data-20


In [ ]:
# !mkdir -p ~/data/
# !gsutil -m cp gs://dcavazos-lyra/weather/data-20/* ~/data/

# 🧠 Train the ML model

https://cloud.google.com/vertex-ai/docs/training/pre-built-containers#pytorch

https://cloud.google.com/vertex-ai/docs/predictions/configure-compute#machine-types

https://pytorch.org/tutorials/beginner/saving_loading_models.html

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html


In [ ]:
from google.cloud import aiplatform

epochs = 100

aiplatform.init(project=project, location=location, staging_bucket=bucket)

job = aiplatform.CustomTrainingJob(
    display_name=f"weather-gpu",
    script_path="trainer/task.py",
    container_uri="us-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-11:latest",
)

job.run(
    machine_type='n1-highmem-8',
    accelerator_type='NVIDIA_TESLA_T4',
    accelerator_count=1,
    args=[
        f"--data-path=/gcs/{bucket}/weather/data",
        f"--model-path=/gcs/{bucket}/weather/model.pt",
        f"--epochs={epochs}",
    ],
)

https://console.cloud.google.com/vertex-ai/training/custom-jobs

In [ ]:
!gsutil ls -lh gs://{bucket}/weather/model.pt

 45.15 KiB  2022-09-15T21:07:23Z  gs://dcavazos-lyra/weather/model.pt
TOTAL: 1 objects, 46231 bytes (45.15 KiB)


In [53]:
!gsutil cp gs://{bucket}/weather/model.pt model.pt

Copying gs://dcavazos-lyra/weather/model.pt...
/ [1 files][ 45.2 KiB/ 45.2 KiB]                                                
Operation completed over 1 objects/45.2 KiB.                                     


# 🔮 Make predictions

In [ ]:
# # Setup some test data to predict   
# test_data = [
#     ['2021-12-23T17:00:00', -118.04194946289063, 40.788608828924076, 'light rain'],    
#     ['2021-12-15T17:30:00', -76.23759765625002, 43.16430423079589, 'medium rain'],          
#     ['2021-12-23T17:00:00', -119.66792602539063, 36.882018532995076, 'heavy rain'],    
#     ['2021-12-23T17:00:00', -112.0310546875, 36.27880268550452, 'just cloudy'],         
#     ['2021-09-23T17:00:00', -119.98515625, 39.68941717505306, 'clear'],           
#     ['2021-11-15T17:30:00', -123.72188110351563, 45.6445363447774, 'medium rain'],         
#     ['2021-12-23T17:00:00', -115.70048828125, 35.324365287778654, 'light rain']
# ]

Now we are ready to execute the model on the test data and get some predictions.

In [42]:
from datetime import datetime

from datasets import Point, get_training_example

date = datetime(2019, 9, 3, 18)
point = Point(26.1, -79.2)
example = get_training_example(date, point, patch_size=128)
print(f"inputs : {example.inputs.dtype} {example.inputs.shape}")
print(f"labels : {example.labels.dtype} {example.labels.shape}")

inputs : float32 (17, 3, 128, 128)
labels : float32 (1, 2, 128, 128)


In [60]:
from trainer.task import Model, Normalize
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

with open("model.pt", "rb") as f:
    model = torch.load(f, device)
    model.eval()
model

Model(
  (layers): Sequential(
    (0): Normalize()
    (1): Conv3d(17, 8, kernel_size=(3, 5, 5), stride=(1, 1, 1))
    (2): ReLU()
    (3): ConvTranspose3d(8, 1, kernel_size=(2, 5, 5), stride=(1, 1, 1))
  )
  (loss): SmoothL1Loss()
)

In [48]:
from visualize import show_inputs, show_labels

show_inputs(example.inputs)

In [61]:
import torch

inputs = torch.from_numpy(example.inputs).to(device)
with torch.no_grad():
    predictions = model(inputs).to("cpu").numpy()

show_labels(example.labels)
show_labels(predictions)

# 5. ⛵ Further exploration

This notebook demonstrated a simple model to start exploring the problem of weather forecasting using deep neural networks. The model has less than 100k parameters and only a few Conv2D layers to keep training time short. Even so, the model is able to distinguish cloud patterns for broad rain vs no rain detection. 

There has been a lot of interesting research work on weather nowcasting recently, especially with [U-Net](https://en.wikipedia.org/wiki/U-Net) style model architectures. If you are interested in diving deeper, here are some articles from Google Research: 

*  [Google Research blog on nowcasting](https://ai.googleblog.com/2021/11/metnet-2-deep-learning-for-12-hour.html)
*  [MetNet paper](https://arxiv.org/abs/2003.12140)

# 6. 🧹 Clean Up

To **avoid incurring charges** to your Google Cloud account for the resources used in this tutorial, either delete the project that contains the resources, or keep the project and delete the individual resources.

## Deleting the project

The **easiest** way to **eliminate billing** is to delete the project that you created for the tutorial.

To delete the project:

> ⚠️ Deleting a project has the following effects:
>
> * **Everything in the project is deleted.** If you used an existing project for this tutorial, when you delete it, you also delete any other work you've done in the project.
>
> * **Custom project IDs are lost.** When you created this project, you might have created a custom project ID that you want to use in the future. To preserve the URLs that use the project ID, such as an appspot.com URL, delete selected resources inside the project instead of deleting the whole project.
>
> If you plan to explore multiple tutorials and quickstarts, **reusing** projects can help you avoid exceeding project **quota limits**.

1. In the Cloud Console, go to the **Manage resources** page.

  <button>

  [Go to Manage resources](https://console.cloud.google.com/iam-admin/projects)

  </button>

1. In the project list, select the project that you want to delete, and then click **Delete**.

1. In the dialog, type the project ID, and then click **Shut down** to delete the project.